In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cufflinks as cf
from sklearn.impute import SimpleImputer
from scipy.stats import ks_2samp
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Configuración inicial
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_rows", 3000)
cf.go_offline()

# 1. Carga y exploración inicial de datos
df = pd.read_csv("/titanic.csv")
print("Forma inicial del dataset:", df.shape)
print("\nDistribución de survived:\n", df.survived.value_counts(normalize=True))

# 2. Análisis de datos faltantes
def completitud_datos_nulos(df):
    return df.isnull().sum().sort_values(ascending=False) / df.shape[0]

print("\nPorcentaje de valores nulos por columna:")
print(completitud_datos_nulos(df))

# 3. Limpieza inicial
df = df[(~df["survived"].isnull()) & (~df["pclass"].isnull()) & (~df["fare"].isnull())]

# 4. Ingeniería de características MEJORADA
df['sex_encoded'] = df['sex'].map({'male': 0, 'female': 1})
df['family_size'] = df['sibsp'] + df['parch'] + 1  # +1 para incluir al pasajero
df['is_alone'] = (df['family_size'] == 1).astype(int)
df['fare_per_person'] = df['fare'] / df['family_size']
df['age_class'] = df['age'] * df['pclass']  # Interacción importante

# 5. Imputación de valores nulos MEJORADA
def complete_continuous_variables(df, cols, strategy='median'):
    imputer = SimpleImputer(strategy=strategy)
    for col in cols:
        if col in df.columns:
            original = df[col].dropna()
            if len(original) > 0:  # Solo imputar si hay valores no nulos
                df[col] = imputer.fit_transform(df[[col]])
                # Verificación KS
                imputed = df.loc[original.index, col]
                ks_stat = ks_2samp(original, imputed).statistic
                print(f"KS-test para {col}: {ks_stat:.4f}")
                if ks_stat >= 0.1:
                    print(f"¡Advertencia: Gran diferencia en distribución para {col}!")
    return df

# Imputar variables importantes
df = complete_continuous_variables(df, ["age", "fare", "body", "fare_per_person"])

# 6. Selección de variables finales
varc = ["pclass", "fare", "age", "sex_encoded", "family_size",
        "is_alone", "fare_per_person", "age_class"]
tgt = "survived"

# 7. Preparación de datos
X = df[varc].copy()
y = df[tgt].copy().astype(int)

# Escalado para modelos que lo requieran
sc = MinMaxScaler()
Xs = pd.DataFrame(sc.fit_transform(X), columns=varc)

# División estratificada MEJORADA
Xt, Xv, yt, yv = train_test_split(Xs, y, train_size=0.7, random_state=42, stratify=y)

# 8. Función de entrenamiento MEJORADA
def entrenar(modelo, param, X, y):
    grid = RandomizedSearchCV(
        estimator=modelo,
        param_distributions=param,
        n_iter=100,  # Más iteraciones para mejor búsqueda
        cv=5,        # Más folds para mejor validación
        scoring='roc_auc',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    grid.fit(X, y)
    print(f"\nMejores parámetros encontrados: {grid.best_params_}")
    print(f"Mejor puntuación AUC-ROC: {grid.best_score_:.4f}")
    return grid.best_estimator_, grid.best_score_, grid.best_params_

# 9. Optimización de Random Forest MEJORADA
param_rf = {
    'n_estimators': range(100, 501, 50),
    'max_depth': range(5, 21),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'class_weight': ['balanced', 'balanced_subsample', None],
    'random_state': [42]
}

print("\nEntrenando Random Forest...")
modelo_rf = RandomForestClassifier()
best_rf, score_rf, params_rf = entrenar(modelo_rf, param_rf, Xt, yt)

# 10. Función de métricas MEJORADA
def metricas(modelo, Xt, Xv, yt, yv):
    probas_train = modelo.predict_proba(Xt)[:, 1]
    probas_val = modelo.predict_proba(Xv)[:, 1]

    return {
        'train_auc': roc_auc_score(yt, probas_train),
        'val_auc': roc_auc_score(yv, probas_val),
        'diff': abs(roc_auc_score(yt, probas_train) - roc_auc_score(yv, probas_val))
    }

# 11. Evaluación final MEJORADA
results_rf = metricas(best_rf, Xt, Xv, yt, yv)
print("\nResultados finales Random Forest:")
print(f"AUC-ROC Entrenamiento: {results_rf['train_auc']:.4f}")
print(f"AUC-ROC Validación: {results_rf['val_auc']:.4f}")
print(f"Diferencia: {results_rf['diff']:.4f}")

# 12. Análisis de overfitting
if results_rf['diff'] > 0.1:
    print("\n¡Advertencia: Posible overfitting! Diferencia > 10%")
elif results_rf['diff'] > 0.05:
    print("\nAdvertencia: Moderada diferencia entre train/validation")
else:
    print("\nEl modelo generaliza bien. Diferencias mínimas.")

# 13. Recomendaciones según resultados
if results_rf['val_auc'] >= 0.85:
    print("\n¡Objetivo alcanzado! Modelo con AUC-ROC >= 85%")
else:
    print("\nRecomendaciones para mejorar:")
    print("- Añadir más variables relevantes (ej: título extraído del nombre)")
    print("- Probar técnicas avanzadas de imputación (KNN imputer)")
    print("- Experimentar con más iteraciones en la búsqueda de hiperparámetros")
    print("- Ajustar balance de clases con técnicas de muestreo (SMOTE)")

Forma inicial del dataset: (1310, 14)

Distribución de survived:
 survived
0.0    0.618029
1.0    0.381971
Name: proportion, dtype: float64

Porcentaje de valores nulos por columna:
body         0.907634
cabin        0.774809
boat         0.629008
home.dest    0.431298
age          0.201527
embarked     0.002290
fare         0.001527
sibsp        0.000763
name         0.000763
survived     0.000763
pclass       0.000763
sex          0.000763
parch        0.000763
ticket       0.000763
dtype: float64
KS-test para age: 0.0000
KS-test para fare: 0.0000
KS-test para body: 0.0000
KS-test para fare_per_person: 0.0000

Entrenando Random Forest...
Fitting 5 folds for each of 100 candidates, totalling 500 fits

Mejores parámetros encontrados: {'random_state': 42, 'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 5, 'class_weight': None, 'bootstrap': False}
Mejor puntuación AUC-ROC: 0.8358

Resultados finales Random Forest:
AUC-ROC Entrenami

In [5]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer

# Configuración para evitar warnings
import warnings
warnings.filterwarnings('ignore')

# 1. Carga y limpieza inicial de datos
df = pd.read_csv("/titanic.csv")

# 2. Ingeniería de características robusta
def feature_engineering(df):
    # Extraer título del nombre
    df['title'] = df['name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    df['title'] = df['title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don',
                                     'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['title'] = df['title'].replace('Mlle', 'Miss')
    df['title'] = df['title'].replace('Ms', 'Miss')
    df['title'] = df['title'].replace('Mme', 'Mrs')

    # Familia y acompañantes
    df['family_size'] = df['sibsp'] + df['parch'] + 1
    df['is_alone'] = (df['family_size'] == 1).astype(int)

    # Manejo seguro de fare_per_person
    df['fare_per_person'] = df['fare'] / df['family_size'].replace(0, 1)  # Evitar división por cero
    df['fare_per_person'] = df['fare_per_person'].replace([np.inf, -np.inf], np.nan)  # Reemplazar infinitos

    # Interacciones importantes
    df['age_class'] = df['pclass'] * df['age'].fillna(df['age'].median())

    # Características de cabina
    df['cabin_known'] = (~df['cabin'].isnull()).astype(int)

    return df

df = feature_engineering(df)

# 3. Selección y limpieza final de variables
cat_features = ['sex', 'embarked', 'title', 'pclass', 'is_alone', 'cabin_known']
num_features = ['age', 'fare', 'family_size', 'fare_per_person', 'age_class']
all_features = cat_features + num_features
tgt = "survived"

# Eliminar filas donde el target es nulo
df = df[~df[tgt].isnull()].copy()
y = df[tgt].astype(int)

# 4. División estratificada
Xt, Xv, yt, yv = train_test_split(df[all_features], y, test_size=0.2, random_state=42, stratify=y)

# 5. Pipeline de preprocesamiento robusto
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', MinMaxScaler()),
    ('selector', SelectKBest(f_classif, k='all'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

# 6. Modelo de red neuronal con configuración robusta
mlp = MLPClassifier(
    early_stopping=True,
    random_state=42,
    max_iter=2000,
    n_iter_no_change=50,
    learning_rate='adaptive'
)

# 7. Pipeline completo
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('mlp', mlp)
])

# 8. Espacio de búsqueda optimizado
param_dist = {
    'mlp__hidden_layer_sizes': [(100, 50), (150, 100, 50), (200, 100)],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__alpha': [0.0001, 0.0005],
    'mlp__learning_rate_init': [0.001, 0.005],
    'mlp__batch_size': [32, 64],
    'preprocessor__num__selector__k': [3, 4, 5]
}

# 9. Búsqueda de hiperparámetros con manejo robusto
search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

print("\nEntrenando red neuronal optimizada...")
search.fit(Xt, yt)

# 10. Evaluación y resultados
best_model = search.best_estimator_

# Función segura para evaluación
def safe_evaluate(model, Xt, Xv, yt, yv):
    try:
        probas_train = model.predict_proba(Xt)[:, 1]
        probas_val = model.predict_proba(Xv)[:, 1]

        return {
            'train_auc': roc_auc_score(yt, probas_train),
            'val_auc': roc_auc_score(yv, probas_val),
            'diff': abs(roc_auc_score(yt, probas_train) - roc_auc_score(yv, probas_val))
        }
    except Exception as e:
        print(f"Error en evaluación: {str(e)}")
        return {'train_auc': 0, 'val_auc': 0, 'diff': 0}

results = safe_evaluate(best_model, Xt, Xv, yt, yv)

print("\n=== Resultados Finales ===")
print(f"AUC-ROC Entrenamiento: {results['train_auc']:.4f}")
print(f"AUC-ROC Validación: {results['val_auc']:.4f}")
print(f"Diferencia: {results['diff']:.4f}")

# 11. Verificación de objetivo
if results['train_auc'] >= 0.85 and results['val_auc'] >= 0.85:
    print("\n¡Éxito: Modelo alcanzó el objetivo en ambos conjuntos!")
elif results['val_auc'] >= 0.85:
    print("\n¡Validación exitosa! Estrategias para mejorar entrenamiento:")
    print("- Aumentar capacidad del modelo (más neuronas/capas)")
    print("- Reducir regularización (disminuir alpha)")
    print("- Aumentar iteraciones de entrenamiento")
else:
    print("\nEstrategias para mejorar:")
    print("- Incrementar n_iter en la búsqueda (50-100)")
    print("- Probar arquitecturas más complejas (más capas ocultas)")
    print("- Añadir más características relevantes")
    print("- Usar técnicas avanzadas de imputación")


Entrenando red neuronal optimizada...
Fitting 5 folds for each of 30 candidates, totalling 150 fits

=== Resultados Finales ===
AUC-ROC Entrenamiento: 0.8525
AUC-ROC Validación: 0.8846
Diferencia: 0.0320

¡Éxito: Modelo alcanzó el objetivo en ambos conjuntos!
